In [17]:
# from get_links import links_on_page
import numpy as np
import pandas as pd
import requests
import re
import networkx as nx
import matplotlib.pyplot as plt
import nltk
import sklearn
from bs4 import BeautifulSoup
from tqdm import tqdm
from animal_list import names_from_table
from netwulf import visualize

In [2]:
animal_names = names_from_table()
animal_df = pd.DataFrame(animal_names.items(), columns=['page-name', 'animal'])


In [3]:
# animal_names
animal_df

,page-name,animal
0,/wiki/Aardvark,Aardvark
1,/wiki/Albatross,Albatross
2,/wiki/Alligator,Alligator
3,/wiki/Alpaca,Alpaca
4,/wiki/Ant,Ant
...,...,...
223,/wiki/Whale,Whale
224,/wiki/Wolf,Wolf
225,/wiki/Wolverine,Wolverine
226,/wiki/Wombat,Wombat


In [4]:
# create function that will run on each row in the dataframe, which will take the page name and return all the readable text on the page

def get_text(page_name):
    url = 'https://en.wikipedia.org' + page_name
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find all the paragraphs in the body
    paragraphs = soup.body.find_all('p')
    # Extract the text from the paragraphs but remove \n
    # text = [p.text for p in paragraphs]
    text = [str(p.text.replace('\n', '')).strip() for p in paragraphs]
    # Join the paragraphs together
    joined_text = ' '.join(text)
    # remove first space
    joined_text = joined_text[1:]
    return joined_text
        

In [5]:
get_text('/wiki/Aardvark')

'See text The aardvark (/ˈɑːrdvɑːrk/ ARD-vark; Orycteropus afer) is a medium-sized, burrowing, nocturnal mammal native to Africa.[2][3] It is the only living species of the order Tubulidentata,[4][5] although other prehistoric species and genera of Tubulidentata are known. Unlike most other insectivores, it has a long snout, similar to that of a pig, which is used to sniff out food. The aardvark is found over much of the southern two-thirds of the African continent, avoiding areas that are mainly rocky. A nocturnal feeder, it subsists on ants and termites, which it will dig out of their hills using its sharp claws and powerful legs. It also digs to create burrows in which to live and rear its young. The animal is listed as "least concern" by the IUCN, although its numbers are decreasing. Aardvarks are afrotheres, a clade which also includes elephants, manatees, and hyraxes. The aardvark is sometimes colloquially called the "African ant bear",[6] "anteater" (not to be confused with the 

In [6]:
# create a new column in the dataframe that will contain the text from the page
animal_df['text'] = animal_df['page-name'].apply(get_text)

In [7]:
def tokenize(text):
    # convert to lowercase
    text = text.lower()

    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # remove urls
    text = re.sub(r'http\S+', '', text)

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # tokenize
    tokens = nltk.word_tokenize(text)

    # remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    return tokens

In [8]:
# create a new column in the dataframe that will contain the tokenized text from the page
animal_df['tokens'] = animal_df['text'].apply(tokenize)

In [9]:
animal_df

,page-name,animal,text,tokens
0,/wiki/Aardvark,Aardvark,See text The aardvark (/ˈɑːrdvɑːrk/ ARD-vark; ...,"[see, text, aardvark, ˈɑːrdvɑːrk, ardvark, ory..."
1,/wiki/Albatross,Albatross,Captured at Midway Atoll DiomedeaThalassarche...,"[captured, midway, atoll, diomedeathalassarche..."
2,/wiki/Alligator,Alligator,"An alligator, or just gator, is a large reptil...","[alligator, gator, large, reptile, crocodilia,..."
3,/wiki/Alpaca,Alpaca,"Camelus pacos Linnaeus, 1758Vicugna pacos (Li...","[camelus, pacos, linnaeus, vicugna, pacos, lin..."
4,/wiki/Ant,Ant,Martialinae Leptanillinae Amblyoponinae Parapo...,"[martialinae, leptanillinae, amblyoponinae, pa..."
...,...,...,...,...
223,/wiki/Whale,Whale,Whales are a widely distributed and diverse gr...,"[whales, widely, distributed, diverse, group, ..."
224,/wiki/Wolf,Wolf,See Subspecies of Canis lupus The wolf (Canis ...,"[see, subspecies, canis, lupus, wolf, canis, l..."
225,/wiki/Wolverine,Wolverine,American wolverine (G. g. luscus) Eurasian wol...,"[american, wolverine, g, g, luscus, eurasian, ..."
226,/wiki/Wombat,Wombat,"Wombats are short-legged, muscular quadrupedal...","[wombats, shortlegged, muscular, quadrupedal, ..."


In [25]:
# td-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(animal_df['text'])
tfidf_matrix.shape



(228, 40889)

In [72]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
vectors = vectorizer.fit_transform(animal_df['text'].iloc[0:10])
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

c:\Users\otto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [73]:
def tf_idf(corpus):
    tf_dist = [nltk.FreqDist(text) for tokens in corpus]


,aarde,aardvark,aardvarklike,aardvarks,abandon,abandoned,abattoirs,abdomen,abdominal,abilities,...,ˈaːjoːtɬ,ˈtoːtʃtɬi,ˈɑːrdvɑːrk,ˈɑːrtfark,μύρμηξ,ώψ,ยำ,ยำไขมดแดง,ἀνθος,ἀνθόλοψ
0,0.016526,0.577685,0.033053,0.319570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.016526,0.016526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.003662,0.000000,0.003662,0.004924,0.000000,0.000000,0.009847,0.029541,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.004924,0.000000,0.004924,0.004924,0.000000,0.000000
5,0.000000,0.048009,0.000000,0.048009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020041,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.011788,0.000000,0.000000,0.011788,0.011788
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023664,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021774,0.021774,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.010107,0.010107,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
